<h1 style="white-space: nowrap" align="center">Mini-Project 1</h1>
<hr style="clear:both"></hr>

In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

Gtk-Message: 14:05:14.351: Failed to load module "canberra-gtk-module"


In [2]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()


(ipykernel_launcher.py:466): Gtk-CRITICAL **: 14:06:33.230: gtk_window_resize: assertion 'height > 0' failed


In [3]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

<h2>DOWNLOAD OF FILES</h2>

In [ ]:
dataset_id = 'ds000171'
subject = 'control01' 

sample_path = "/home/jovyan/Data/dataset"

mkdir_no_exist(sample_path)
bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

mkdir_no_exist(bids_root)

subject_dir = 'sub-{}'.format(subject)

In [4]:
###################
# Openneuro download.
###################
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", op.join(subject_dir, '*'),# We are asking to get all files within the subject_dirt folder by using the wildcard *
               ], check=True)

###################
# Create folders relevant for preprocessing.
# In BIDs, ANYTHING we modify must go in the derivatives folder, to keep original files clean in case we make a mistake.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-control01'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'fmap'))


👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds000171 …


📁 Traversing directories for ds000171 : 436 entities [00:50,  8.70 entities/s]


📥 Retrieving up to 15 files (5 concurrent downloads). 


sub-control01_T1w.nii.gz:   0%|          | 0.00/8.77M [00:00<?, ?B/s]
dataset_description.json:   0%|          | 0.00/2.79k [00:00<?, ?B/s]
                                                                     
CHANGES:   0%|          | 0.00/45.0 [00:00<?, ?B/s]
                                                   
README:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

participants.tsv:   0%|          | 0.00/1.63k [00:00<?, ?B/s]
                                                   

sub-control01_T1w.nii.gz:   3%|▎         | 287k/8.77M [00:00<00:07, 1.16MB/s]
sub-control01_T1w.nii.gz:   5%|▌         | 492k/8.77M [00:00<00:05, 1.52MB/s]
                                                              
sub-control01_task-music_run-2_events.tsv:   0%|          | 0.00/339 [00:00<?, ?B/s]
sub-control01_T1w.nii.gz:  13%|█▎        | 1.11M/8.77M [00:00<00:02, 3.31MB/s]      
sub-control01_task-music_run-2_bold.nii.gz:   0%|          | 0.00/33.9M [00:00<?, ?B/s]

sub-control01_T1w.nii.gz:  33%|███▎     

✅ Finished downloading ds000171.
 
🧠 Please enjoy your brains.
 


<h3>Function used to clear the directory in case the process compromises the data and we need to download everything again :</h3>

In [5]:
def clear_data(basis = True, deriv = True, anat = True, func = True, mask = False):
    if basis:
        os.remove(op.join(bids_root, 'README'))
        os.remove(op.join(bids_root, 'CHANGES'))
        os.remove(op.join(bids_root, 'participants.tsv'))
        os.remove(op.join(bids_root, 'dataset_description.json'))
    if mask:
        os.remove(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))
        os.remove(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-control01', 'anat', 'sub-control01_T1w_mask.nii.gz'))
    if deriv:
        os.rmdir(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-control01', 'anat'))
        os.rmdir(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-control01', 'fmap'))
        os.rmdir(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-control01', 'func'))
    if anat:
        os.remove(op.join(bids_root, 'sub-control01','anat', 'sub-control01_T1w.nii.gz'))
        os.rmdir(op.join(bids_root, 'sub-control01', 'anat'))
    if func:
        os.remove(op.join(bids_root,'sub-control01','func', 'sub-control01_task-music_run-1_bold.nii.gz'))
        os.remove(op.join(bids_root, 'sub-control01','func', 'sub-control01_task-music_run-2_bold.nii.gz'))
        os.remove(op.join(bids_root,'sub-control01','func', 'sub-control01_task-music_run-3_bold.nii.gz'))
        os.remove(op.join(bids_root,  'sub-control01','func', 'sub-control01_task-nonmusic_run-4_bold.nii.gz'))
        os.remove(op.join(bids_root, 'sub-control01','func', 'sub-control01_task-nonmusic_run-5_bold.nii.gz'))
        os.remove(op.join(bids_root,  'sub-control01','func', 'sub-control01_task-music_run-1_events.tsv'))
        os.remove(op.join(bids_root, 'sub-control01','func', 'sub-control01_task-music_run-2_events.tsv'))
        os.remove(op.join(bids_root, 'sub-control01','func', 'sub-control01_task-music_run-3_events.tsv'))
        os.remove(op.join(bids_root,'sub-control01','func', 'sub-control01_task-nonmusic_run-4_events.tsv'))
        os.remove(op.join(bids_root, 'sub-control01','func', 'sub-control01_task-nonmusic_run-5_events.tsv'))
        os.rmdir(op.join(bids_root, 'sub-control01','func'))
    os.rmdir(op.join(bids_root, 'sub-control01'))
    os.rmdir(op.join(bids_root, 'derivatives', 'preprocessed_data', 'sub-001'))
    os.rmdir(op.join(bids_root, 'derivatives', 'preprocessed_data'))
    os.rmdir(op.join(bids_root, 'derivatives'))

In [6]:
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv
|--------- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-4_events.tsv
|--------- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-5_events.tsv


In [ ]:
patient_path = op.join(bids_root, 'sub-control01')
anatomical_path = op.join(patient_path, 'anat')
anatomical_file = op.join(anatomical_path, 'sub-control01_T1w.nii.gz')
functional_path = op.join(patient_path, 'func')

<h2>Skull Stripping (Here robust method is used)</h2>

In [ ]:
def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------result
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    subject = 'sub-{}'.format(subject_id)
    anat_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
    betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
    os.system('bet {} {} -m {}'.format(anat_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_mask')
get_skull_stripped_anatomical(bids_root, preproc_root, "control01", True)

In [9]:
## TO VERIFY IF MASK WAS PROPERLY GENERATED
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv
|--------- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-4_events.tsv
|--------- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-5_events.tsv


In [10]:
resulting_treated_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz')make sure to standardize
each session before concatenation!

In [11]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anatomical_file)
fsleyesDisplay.load(resulting_mask_path)


(ipykernel_launcher.py:424): Gdk-WARNING **: 12:54:34.319: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [12]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(resulting_treated_path)

In [14]:
def fsl_mask(img_path, mask_path, masked_img_path):
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))

In [16]:
## IN CASE YOU WANT TO EDIT THE MASK THAT WAS GENERATED BY BET FEEL FREE TO SKIP IT ROBUST ALGORITHM SEEMS TO WORK PRETTY WELL
fsl_mask(anatomical_file, resulting_mask_path, resulting_treated_path)

In [13]:
functional_first_path = op.join(functional_path, 'sub-control01_task-music_run-1_bold.nii.gz')

In [14]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(functional_path, 'sub-control01_task-music_run-1_bold.nii.gz'))

<h3>Use of FLIRT for alignment and organization of views</h3>

In [15]:
from fsl.wrappers import flirt
subject_id = 'control01'
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
target = resulting_treated_path # Fill me
reference = mni_template # Fill me
result = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni'.format(subject_id))
flirt(target, reference, out=result)


Final result: 
0.003348 0.005721 -1.116064 204.794610 
-0.933350 0.500887 0.005750 178.784833 
0.501418 1.048657 -0.003251 -149.700516 
0.000000 0.000000 0.000000 1.000000 



{}

In [18]:
Error: cant open file /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle.nii.gz
Running FAST segmentation
terminate called after throwing an instance of 'NiftiIO::NiftiException'
  what():  Error: cant open file /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_seg.nii.gz
/opt/fsl-6.0.7.4/bin/epi_reg: line 314:  3593 Aborted                 (core dumped) $FSLDIR/bin/fast -o ${vout}_fast ${vrefbrain}
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_pve_2
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_pve_2
/opt/fsl-6.0.7.4/bin/epi_reg: line 314:  3873 Aborted                 (core dumped) $FSLDIR/bin/fslmaths ${vout}_fast_pve_2 -thr 0.5 -bin ${vout}_fast_wmseg
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_wmseg
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_wmseg
/opt/fsl-6.0.7.4/bin/epi_reg: line 320:  3875 Aborted                 (core dumped) $FSLDIR/bin/fslmaths ${vout}_fast_wmseg -edge -bin -mas ${vout}_fast_wmseg ${vout}_fast_wmedge
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Image Exception : #22 :: Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Image Exception : #22 :: Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Could not open matrix file /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr.mat
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Image Exception : #22 :: Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle
Error : No image files match: /home/jovyan/Data/dataset/ds000171/derivaprint_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|--------------- sub-control01_T1w_mni.nii.gz
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv
|--------- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-4_events.tsv
|--------- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-5_events.tsv


In [19]:
## VERIFY IF THE PROCESS HAPPENED CORRECTLY
processed_volume = result
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(processed_volume)

<h2>Movement correction of motion corrected fMRI using MCFLIRT</h2>

In [20]:
outpath = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco')

In [21]:
from fsl.wrappers import mcflirt
mcflirt(infile=functional_first_path,o=outpath, plots=True, report=True, dof=6, mats=True)

Processed data will be saved as /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control01/func/sub-control01_task-music_run-1_bold_moco

McFLIRT v 2.0 - FMRI motion correction

Reading time series... 
first iteration - 8mm scaling, set tolerance
Rescaling reference volume [52] to 8 mm pixels
Registering volumes ... [53][54][55][56][57][58][59][60][61][62][63][64][65][66][67][68][69][70][71][72][73][74][75][76][77][78][79][80][81][82][83][84][85][86][87][88][89][90][91][92][93][94][95][96][97][98][99][100][101][102][103][104][51][50][49][48][47][46][45][44][43][42][41][40][39][38][37][36][35][34][33][32][31][30][29][28][27][26][25][24][23][22][21][20][19][18][17][16][15][14][13][12][11][10][9][8][7][6][5][4][3][2][1][0]
second iteration - drop to 4mm scaling
Rescaling reference volume [52] to 4 mm pixels
Registering volumes ... [53][54][55][56][57][58][59][60][61][62][63][64][65][66][67][68][69][70][71][72][73][74][75][76][77][78][79][80][81][82][83][84][85][86][87]

refnum = 52
Original_refvol = -1


Registering volumes ... [53][54][55][56][57][58][59][60][61][62][63][64][65][66][67][68][69][70][71][72][73][74][75][76][77][78][79][80][81][82][83][84][85][86][87][88][89][90][91][92][93][94][95][96][97][98][99][100][101][102][103][104][51][50][49][48][47][46][45][44][43][42][41][40][39][38][37][36][35][34][33][32][31][30][29][28][27][26][25][24][23][22][21][20][19][18][17][16][15][14][13][12][11][10][9][8][7][6][5][4][3][2][1][0]
Saving motion corrected time series... 


{}

In [33]:
print_dir_tree(bids_root, max_depth=6)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|--------------- sub-control01_T1w_mni.nii.gz
|------------ fmap/
|------------ func/
|--------------- sub-control01_task-music_run-1_bold_moco.nii.gz
|--------------- sub-control01_task-music_run-1_bold_moco.par
|--------------- sub-control01_task-music_run-1_bold_moco.mat/
|------------------ MAT_0000
|------------------ MAT_0001
|------------------ MAT_0002
|------------------ MAT_0003
|------------------ MAT_0004
|------------------ MAT_0005
|------------------ MAT_0006
|------------------ MAT_0007
|------------------ MAT_0008
|------------------ MAT_0009
|------------------ MAT_0010
|------------------ MAT_0011
|------------------ MAT_0012
|------------------ MAT_0013
|------------------ MAT_0014
|------------------ M

In [24]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(outpath)

<h2>Coregistration of fMRI data using EPI-REG</h2>

In [31]:
skull_stripped_t1 = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w')
output_path = op.join(preproc_root, 'sub-control-01', 'func', 'sub-control01_task-music_run-1_bold_bbr')
ref_vol_name =  op.join(preproc_root, 'sub-001', 'func', 'sub-control01_task-music_run-01_bold_vol_middle')

In [25]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(skull_stripped_t1)

In [43]:
from fsl.wrappers import epi_reg
fslroi(outpath, ref_vol_name, str(52), str(1))
subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(anatomical_file), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])

Error: cant open file /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle.nii.gz


Running FAST segmentation


terminate called after throwing an instance of 'NiftiIO::NiftiException'
  what():  Error: cant open file /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_seg.nii.gz
/opt/fsl-6.0.7.4/bin/epi_reg: line 314:  3593 Aborted                 (core dumped) $FSLDIR/bin/fast -o ${vout}_fast ${vrefbrain}
Image Exception : #63 :: No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_pve_2
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: /home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr_fast_pve_2
/opt/fsl-6.0.7.4/bin/epi_reg: line 314:  3873 Aborted                 (core dumped) $FSLDIR/bin/fslmaths ${vout}_fast_pve_2 -thr 0.5 -bin ${vout}_fast_wmseg
Image Exception : #63 :: No image files match:

FLIRT pre-alignment
Running BBR


CompletedProcess(args=['epi_reg', '--epi=/home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-001/func/sub-control01_task-music_run-1_bold_vol_middle', '--t1=/home/jovyan/Data/dataset/ds000171/sub-control01/anat/sub-control01_T1w.nii.gz', '--t1brain=/home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control01/anat/sub-control01_T1w', '--out=/home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr'], returncode=0)

In [ ]:
subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(anatomical_file), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])

In [27]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(skull_stripped_t1)
fsleyesDisplay.load(output_path)

PathError: Could not find a supported file with prefix "/home/jovyan/Data/dataset/ds000171/derivatives/preprocessed_data/sub-control-01/func/sub-control01_task-music_run-1_bold_bbr"